In [ ]:
import os
import sys
import glob
import logging
import fnmatch
import subprocess
from multiprocessing import Process, cpu_count

from os.path import expanduser
home = expanduser("~")

from ipyfilechooser import FileChooser

SlicerExecutable = "/Applications/Slicer-5.0.2_stable.app/Contents/MacOS/Slicer"

# SlicerExecutable = "/home/herzc/Slicer-4.13.0-2021-07-14-linux-amd64/Slicer"

script_path = os.path.abspath(os.path.join(os.getcwd(), "..", "scripts", "normalize_and_export.py"))
assert os.path.exists(SlicerExecutable), f"{SlicerExecutable} doesn't exist"
assert os.path.exists(script_path), f"{script_path} doesn't exist"


commissural_landmarks = {
    "mitral": ["ALC", "PMC"],
    "tricuspid": ["ASC", "APC", "PSC"],
    "cavc": ["SRC", "SLC", "ILC", "IRC"]
}


In [ ]:
input_dir_selector = FileChooser(os.path.join(home, "Documents"))
display(input_dir_selector)

In [ ]:
output_dir_selector = FileChooser(input_dir_selector.selected_path)
display(output_dir_selector)

In [ ]:
def getAllFilesWithExtension(directory, extension='.nii.gz', fname_only=False):
    files = []
    for root, dirnames, filenames in os.walk(directory):
        for filename in fnmatch.filter(filenames, '*{}'.format(extension)):
            files.append(filename if fname_only else os.path.join(root, filename))
    return files


def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]
            
def run(listing, num_parallel_processes, params, execute_method):
    processed = 0
    
    for files in batch(listing, num_parallel_processes):
        procs = []  
        for f in files:
            p = execute_method(f, **params)
            if p:
                procs.append(p)
            else:
                print(f"Skipping {f}")

        for p in procs:
            p.join()

    print(len(files))
    processed += len(files)
    print("Processed %d%%" % (processed/(len(listing)) * 100.0))
    print("Done!")
    
    
def spawnSlicer(*args, **kwargs):
    logfile = kwargs.get('logfile')
    print(logfile)
    with open(logfile, "w") as f:
        subprocess.run(args, stdout=f, stderr=f, timeout=600)
    
    
def run_normalize_and_export(inputData, 
                             outputDir, 
                             valveType, 
                             referencePhase,
                             cardiacPhaseFrames, 
                             minVoxelHeight, 
                             voxelSpacing,
                             volumeDimensions, 
                             landmarkLabels,
                             exportLandmarks,
                             exportSegmentation,
                             oneFilePerSegment,
                             segmentationPhases,
                             annulusAsLabel,
                             annulusAsModel,
                             runQuantification,
                             annulusPhases,
                             landmarkLabelPhases,
                             debug):
    
    logDirectory = os.path.join(outputDir, "logs")
    if not os.path.exists(logDirectory):
        os.makedirs(logDirectory)
    logfile = os.path.join(logDirectory, "normalize_and_export_{}.log".format(os.path.basename(inputData)))

    args = [SlicerExecutable, "--no-splash", "--python-script", script_path]
    
    if inputData:
        args.extend(["-i", f"{inputData}"])
    if outputDir:
        args.extend(["-o", f"{outputDir}"])
    if valveType:
        args.extend(["-t", f"{valveType}"])
    if referencePhase:
        args.extend(["-rp", f"{referencePhase}"])
    if minVoxelHeight:
        args.append("-vh")
        args.append(f"{minVoxelHeight}")
    if cardiacPhaseFrames:
        args.append("-ph")
        args.extend([phase for phase in cardiacPhaseFrames])
    if volumeDimensions:
        args.append("-dim")
        args.extend([str(r) for r in volumeDimensions])
    if voxelSpacing:
        args.append("-vx")
        args.append(str(voxelSpacing))
    if landmarkLabels:
        args.append("-ll")
        args.extend(landmarkLabels)
    if exportLandmarks:
        args.append("-l")
    if exportSegmentation:
        args.append("-s")
    if oneFilePerSegment:
        args.append("-sf")
    if runQuantification:
        args.append("-qf")
    if annulusAsLabel:
        args.append("-al")
    if annulusAsModel:
        args.append("-am")
    if annulusPhases:
        args.append("-ap")
        args.extend([phase for phase in annulusPhases])
    if segmentationPhases:
        args.append("-sp")
        args.extend([phase for phase in segmentationPhases])
    if landmarkLabelPhases:
        args.append("-llp")
        args.extend([phase for phase in landmarkLabelPhases])
    
        
    if debug:
        args.append("-d")
        
#     print(" ".join(args))
    p = Process(target=spawnSlicer, args=args, kwargs={"logfile": f"{logfile}"})
    p.start()
    return p

In [ ]:
listing = getAllFilesWithExtension(input_dir_selector.selected_path, '.mrb')
len(listing)

## only export the ones that have FPS > 20

In [ ]:
# import pandas as pd
# df_seq = pd.read_csv("/Users/herzc/Documents/CHOP/DeepLearning/Deep_Learning_Datasets/Tricuspid/DeepLearning_Sequence_Info.csv")
# df = pd.read_csv("/Users/herzc/Documents/CHOP/DeepLearning/Deep_Learning_Datasets/Tricuspid/DeepLearning_Frames.csv")
# # del df["Comments"]
# df = df_seq.merge(df, how="left", on="File Name").dropna()
# df["FPS"] = pd.to_numeric(df['FPS'])
# df = df[df["FPS"] > 20.0]

# valid = df["File Name"].tolist()

# listing = [f for f in listing if os.path.basename(f).replace(".mrb", "") in valid]

# len(listing)

## Export 

In [ ]:
valveType = "tricuspid"

num_parallel_processes = 10 #cpu_count()
parameters = dict(
    outputDir = output_dir_selector.selected_path,
    valveType = valveType,
    volumeDimensions = [224] * 3,
    minVoxelHeight = 6,
    voxelSpacing = 0.25,
    landmarkLabels = commissural_landmarks[valveType], 
    exportLandmarks = True,
    exportSegmentation = True, 
    oneFilePerSegment = False,
    annulusAsLabel = True,
    annulusAsModel = True,
    referencePhase = "MS",
    cardiacPhaseFrames = ["MS", "MS+1"],
    segmentationPhases = ["MS", "MS+1"],
    annulusPhases = ["MS", "MS+1"],
    landmarkLabelPhases = ["MS"],
    runQuantification = False,
    debug = False
)

run(listing, num_parallel_processes, params=parameters, execute_method=run_normalize_and_export)